# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint


# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load city weather csv
# Create city weather dataframe and drop any duplicates
city_weather_df = pd.read_csv('../cities.csv')
city_weather_df.dropna(inplace = True)
city_weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53.15,-70.92,33.80,74,75,5.82,CL,1594927232
1,Ribeira Grande,38.52,-28.70,71.60,73,75,4.70,PT,1594927252
2,Sapouy,11.55,-1.77,76.28,84,3,6.15,BF,1594927252
3,Bairiki Village,1.33,172.98,80.60,78,75,4.70,KI,1594927252
4,Kapaa,22.08,-159.32,80.60,78,20,10.29,US,1594927000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]

# convert to float
rating = city_weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure('TERRAIN')

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 )
#set max intensity to 100
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#modify datafrome to only include cities with preferred weather conditions
ideal_weather_df = city_weather_df.loc[((city_weather_df["Max Temp"] > 65) & (city_weather_df["Max Temp"] < 75)) &
                                        (city_weather_df["Humidity"] < 55) & (city_weather_df["Wind Speed"] < 10)
                                      ]

print(ideal_weather_df)

ideal_weather_df.dtypes

                City    Lat     Lng  Max Temp  Humidity  Cloudiness  \
16          Jinchang  38.50  102.17     66.94        51          70   
20       Yellowknife  62.46 -114.35     66.00        48          75   
229            Lluta -16.01  -72.01     73.40         9           0   
281          Córdoba -31.41  -64.18     68.00        32           0   
303  Haines Junction  60.75 -137.51     66.79        52          31   
326       Mossamedes -15.20   12.15     69.26        49           0   
445            Homer  59.64 -151.55     66.20        52           1   
455         Santa Fe -31.63  -60.70     68.00        38           5   
563   Miguel Hidalgo  19.42  -99.20     73.40        38          20   

     Wind Speed Country        Date  
16         5.21      CN  1594927254  
20         4.70      CA  1594927254  
229        9.17      PE  1594927274  
281        6.93      AR  1594927150  
303        1.48      CA  1594926982  
326        3.83      AO  1594927284  
445        9.17      US

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
#create dataframe that will be used to add Nearest Hotel column
hotel_df = ideal_weather_df[["City", "Lat", "Lng", "Country"]].copy()

hotel_df

,City,Lat,Lng,Country
16,Jinchang,38.50,102.17,CN
20,Yellowknife,62.46,-114.35,CA
229,Lluta,-16.01,-72.01,PE
281,Córdoba,-31.41,-64.18,AR
303,Haines Junction,60.75,-137.51,CA
326,Mossamedes,-15.20,12.15,AO
445,Homer,59.64,-151.55,US
455,Santa Fe,-31.63,-60.70,AR
563,Miguel Hidalgo,19.42,-99.20,MX


In [6]:
#create new column place holder for nearest hotel
hotel_df['Nearest Hotel'] = ""

#find the closest hotel to each set of coordinates
target_radius = 5000
place_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type": place_type,
    "radius": target_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    coords = f'{lat}, {lng}'
    
    # add location to params dict
    params['location'] = coords

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {coords}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #print out closest hotel name for each set of coordinates and add it to the hotel dataframe
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Nearest Hotel'] = results[0]['name']
    
    #exception handling for missing hotel names
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

#drop rows with no hotel from the hotel dataframe
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace =True)
hotel_df.dropna(subset = ["Nearest Hotel"], inplace = True)
print(hotel_df)

Retrieving Results for Index 16: 38.5, 102.17.
Closest hotel is Mingshihui Jinchang International Hotel.
------------
Retrieving Results for Index 20: 62.46, -114.35.
Closest hotel is The Explorer Hotel.
------------
Retrieving Results for Index 229: -16.01, -72.01.
Missing field/result... skipping.
------------
Retrieving Results for Index 281: -31.41, -64.18.
Closest hotel is Sheraton Cordoba Hotel.
------------
Retrieving Results for Index 303: 60.75, -137.51.
Closest hotel is Parkside Inn.
------------
Retrieving Results for Index 326: -15.2, 12.15.
Closest hotel is Hotel Chik Chik Namibe.
------------
Retrieving Results for Index 445: 59.64, -151.55.
Closest hotel is Beluga Lake Lodge.
------------
Retrieving Results for Index 455: -31.63, -60.7.
Closest hotel is Conquistador Hotel.
------------
Retrieving Results for Index 563: 19.42, -99.2.
Closest hotel is JW Marriott Hotel Mexico City.
------------
                City    Lat     Lng Country  \
16          Jinchang  38.50  102

In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig = gmaps.figure()
fig.add_layer(marker_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))